In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math

In [2]:
df=pd.read_csv(r'CakalangTuna (1).csv', sep=',', header=0, low_memory=False, index_col=['tanggal'])
df.head()

,cakalang_surabayakota,cakalang_malangkota,cakalang_banyuwangikab,cakalang_blitarkab,cakalang_jombangkab,cakalang_madiunkab,cakalang_malangkab,cakalang_mojokertokab,cakalang_nganjukkab,cakalang_pasuruankab,...,tuna_lumajangkab,tuna_madiunkab,tuna_malangkab,tuna_mojokertokab,tuna_nganjukkab,tuna_pamekasankab,tuna_pasuruankab,tuna_sidoarjokab,tuna_batukota,tuna_madiunkota
tanggal,,,,,,,,,,,,,,,,,,,,,
2018-05-24,29500,28800,28000,25000,27500,30000,30200,34500,29500,24833,...,31000,32000,34400,37000,35666,70000,29666,32000,38500,34333
2018-05-25,29500,28800,28000,25000,27500,30000,31160,34500,29500,24833,...,31000,32000,34800,37000,35666,70000,29666,32000,38250,34333
2018-05-26,29500,28800,28000,25000,27500,30000,30600,34500,29500,24833,...,31500,32000,35000,37000,35666,70000,29666,32000,38250,34333
2018-05-27,29500,28800,28000,25000,27500,30000,30600,34500,29500,24833,...,31000,32000,35000,37000,35666,70000,29666,32000,38250,34333
2018-05-28,29500,28800,28000,25000,27500,30000,30600,34500,29500,24833,...,31000,32000,35400,37000,35666,70000,29666,32000,38750,34333


In [3]:
# df1=np.array(df[:][:180])
# df_reshaped = np.reshape(df1, (1, 180, 27))
# print(df_reshaped)

In [4]:
df1=df
scalers={}
for i in df.columns:
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(df1[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    df1[i]=s_s

In [5]:
train,test = df1[:math.floor(0.75*1827)], df1[math.floor(0.75*1827):]

In [6]:
# train = train_df
# scalers={}
# for i in train_df.columns:
#     scaler = MinMaxScaler(feature_range=(0,1))
#     s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
#     s_s=np.reshape(s_s,len(s_s))
#     scalers['scaler_'+ i] = scaler
#     train[i]=s_s
# test = test_df
# for i in train_df.columns:
#     scaler = scalers['scaler_'+i]
#     s_s = scaler.transform(test[i].values.reshape(-1,1))
#     s_s=np.reshape(s_s,len(s_s))
#     scalers['scaler_'+i] = scaler
#     test[i]=s_s
# print(train)

In [7]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [8]:
n_past = 180
n_future =  30
n_features = 27
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
# print(X_test.shape[0],X_test.shape[1],X_test.shape[2])
# print(y_test.shape[0],y_test.shape[1],y_test.shape[2])
# print(X_test)

In [9]:
# def create_uncompiled_model():

#     ### START CODE HERE
    
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Conv1D(filters=8, kernel_size=4,
#                                strides=1,
#                                activation='relu',
#                                padding='causal',
#                                input_shape=[None, 27]),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.LSTM(8, return_sequences=True),
#         tf.keras.layers.LSTM(4, return_sequences=True),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(8, activation='relu')),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(27))
#     ])

    
#     ### END CODE HERE

#     return model

In [10]:
# def adjust_learning_rate(X_train, y_train):
    
#     model = create_uncompiled_model()
    
#     lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    
#     ### START CODE HERE
    
#     # Select your optimizer
#     optimizer = tf.keras.optimizers.Adam()
    
#     # Compile the model passing in the appropriate loss
#     model.compile(loss=tf.keras.losses.Huber(),
#                   optimizer=optimizer, 
#                   metrics=["mae"]) 
    
#     ### END CODE HERE
    
#     history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_schedule])
    
#     return history

In [11]:

# lr_history = adjust_learning_rate(X_train,y_train)

In [12]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))

# Add Conv1D layer
encoder_cnn = tf.keras.layers.Conv1D(16, kernel_size=4, activation='relu', padding='causal')(encoder_inputs)

encoder_l1 = tf.keras.layers.LSTM(16, return_sequences=True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_cnn)
encoder_states1 = encoder_outputs1[1:]

encoder_l2 = tf.keras.layers.LSTM(8, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

dropout_rate = 0.4
dropout_layer = tf.keras.layers.Dropout(dropout_rate)
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(dropout_layer(encoder_outputs2[0]))

decoder_l1 = tf.keras.layers.LSTM(16, return_sequences=True)(decoder_inputs, initial_state=encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(8, return_sequences=True)(decoder_l1, initial_state=encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)

model_e2d2 = tf.keras.models.Model(encoder_inputs, decoder_outputs2)

model_e2d2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180, 27)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 180, 16)      1744        ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 180, 16),    2112        ['conv1d[0][0]']                 
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                              

In [13]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
# model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
# history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0089), loss=tf.keras.losses.Huber(),metrics=["mae"])
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=32,verbose=1)#,callbacks=[reduce_lr])

Epoch 1/50
37/37 [==============================] - 12s 151ms/step - loss: 0.0276 - mae: 0.1681 - val_loss: 0.0690 - val_mae: 0.3044
Epoch 2/50
37/37 [==============================] - 4s 114ms/step - loss: 0.0114 - mae: 0.1117 - val_loss: 0.0667 - val_mae: 0.2949
Epoch 3/50
37/37 [==============================] - 5s 135ms/step - loss: 0.0105 - mae: 0.1077 - val_loss: 0.0587 - val_mae: 0.2756
Epoch 4/50
37/37 [==============================] - 5s 137ms/step - loss: 0.0081 - mae: 0.0925 - val_loss: 0.0550 - val_mae: 0.2677
Epoch 5/50
37/37 [==============================] - 5s 133ms/step - loss: 0.0068 - mae: 0.0796 - val_loss: 0.0551 - val_mae: 0.2701
Epoch 6/50
37/37 [==============================] - 5s 138ms/step - loss: 0.0063 - mae: 0.0756 - val_loss: 0.0539 - val_mae: 0.2675
Epoch 7/50
37/37 [==============================] - 5s 134ms/step - loss: 0.0058 - mae: 0.0711 - val_loss: 0.0533 - val_mae: 0.2642
Epoch 8/50
37/37 [==============================] - 5s 133ms/step - loss: 0

In [14]:
pred_e2d2=model_e2d2.predict(X_test)

8/8 [==============================] - 2s 39ms/step


In [15]:
for index,i in enumerate(train.columns):
    scaler = scalers['scaler_'+i]
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index]).astype(int)
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index]).astype(int)
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index]).astype(int)


In [16]:
from sklearn.metrics import mean_absolute_error
print(pred_e2d2)
# for index,i in enumerate(train.columns):
#   print(i)
#   for j in range(1,31):
#     print("Day ",j,":",pred_e2d2[-1,j-1,index])
#     print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index]))
#   print()
# 1,3,2,...
# 1,2,3,...
# 1,2,3,...

# [[12,34,56,...],
#  [13,15,16,...]],
# [[]]

[[[35536. 32504. 29702. ... 35054. 38593. 35499.]
  [35721. 32488. 29815. ... 35053. 38442. 35499.]
  [35688. 32467. 29812. ... 35067. 38533. 35497.]
  ...
  [35649. 32423. 29822. ... 35057. 38613. 35497.]
  [35635. 32423. 29815. ... 35055. 38617. 35498.]
  [35617. 32424. 29807. ... 35054. 38621. 35500.]]

 [[35535. 32500. 29705. ... 35052. 38592. 35498.]
  [35718. 32484. 29815. ... 35051. 38440. 35497.]
  [35687. 32464. 29813. ... 35065. 38532. 35495.]
  ...
  [35648. 32423. 29822. ... 35056. 38613. 35497.]
  [35634. 32423. 29815. ... 35055. 38617. 35498.]
  [35617. 32424. 29807. ... 35054. 38621. 35500.]]

 [[35536. 32495. 29708. ... 35050. 38593. 35497.]
  [35716. 32479. 29816. ... 35048. 38438. 35495.]
  [35685. 32460. 29814. ... 35063. 38530. 35494.]
  ...
  [35648. 32423. 29822. ... 35056. 38613. 35497.]
  [35634. 32423. 29815. ... 35055. 38617. 35498.]
  [35616. 32423. 29807. ... 35053. 38621. 35500.]]

 ...

 [[35653. 32649. 29690. ... 35142. 38678. 35555.]
  [35809. 32637. 297

In [17]:
# import time
# saved_model_path = "model/"

# tf.saved_model.save(model_e2d2, saved_model_path)
# tfjs.converters.convert_tf_saved_model(saved_model_path, saved_model_path)
import time
saved_model_path = "model/{}.h5".format(int(time.time()))

model_e2d2.save(saved_model_path)

In [18]:
# def create_uncompiled_model():

#     ### START CODE HERE
    
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Conv1D(filters=8, kernel_size=4,
#                                strides=1,
#                                activation='relu',
#                                padding='causal',
#                                input_shape=[n_past, n_features]),
#         tf.keras.layers.Dropout(0.4),
#         # tf.keras.layers.LSTM(512, return_sequences=True),
#         # # tf.keras.layers.LSTM(256),
#         # tf.keras.layers.Conv1D(filters=512, kernel_size=4,
#         #                        strides=1,
#         #                        activation='relu',
#         #                        padding='causal'),
#         # tf.keras.layers.LSTM(256, return_sequences=True),
#         # # tf.keras.layers.LSTM(128),
#         # tf.keras.layers.Conv1D(filters=256, kernel_size=4,
#         #                        strides=1,
#         #                        activation='relu',
#         #                        padding='causal'),
#         # tf.keras.layers.LSTM(128, return_sequences=True),
#         # # tf.keras.layers.LSTM(64),
#         # tf.keras.layers.Conv1D(filters=512, kernel_size=4,
#         #                        strides=1,
#         #                        activation='relu',
#         #                        padding='causal'),
#         # tf.keras.layers.LSTM(256, return_sequences=True),
#         # # tf.keras.layers.LSTM(128),
#         # tf.keras.layers.Conv1D(filters=64, kernel_size=4,
#         #                        strides=1,
#         #                        activation='relu',
#         #                        padding='causal'),
#         # tf.keras.layers.LSTM(128, return_sequences=True),
#         # # tf.keras.layers.LSTM(64),
#         # tf.keras.layers.Conv1D(filters=32, kernel_size=4,
#         #                        strides=1,
#         #                        activation='relu',
#         #                        padding='causal'),
#         tf.keras.layers.LSTM(8, return_sequences=True),
#         tf.keras.layers.LSTM(4),
#         tf.keras.layers.Dropout(0.4),                                        
#         # tf.keras.layers.Dense(512, activation='relu'),
#         # tf.keras.layers.Dense(256, activation='relu'),
#         # tf.keras.layers.Dense(256, activation='relu'),
#         # tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dense(8, activation='relu'),
#         tf.keras.layers.Dropout(0.4),                
#         tf.keras.layers.Dense(27)
#     ]) 
    
#     ### END CODE HERE

#     return model

In [19]:
# def adjust_learning_rate(X_train, y_train):
    
#     model = create_uncompiled_model()
    
#     lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    
#     ### START CODE HERE
    
#     # Select your optimizer
#     optimizer = tf.keras.optimizers.Adam()
    
#     # Compile the model passing in the appropriate loss
#     model.compile(loss=tf.keras.losses.Huber(),
#                   optimizer=optimizer, 
#                   metrics=["mae"]) 
    
#     ### END CODE HERE
    
#     history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_schedule])
    
#     return history

In [20]:
# lr_history = adjust_learning_rate(X_train,y_train)